In [38]:
# !pip install mediapipe opencv-python pyttsx3

In [39]:
import cv2
import mediapipe as mp
import numpy as np
import pyttsx3  # Text-to-speech
import threading

mp_drawing = mp.solutions.drawing_utils
mp_body = mp.solutions.pose

In [40]:
engine = pyttsx3.init()
def speak(text):
    thread = threading.Thread(target=lambda: engine.say(text) or engine.runAndWait())
    thread.start()

In [41]:
# initialize webcam
webcam = cv2.VideoCapture(0)

# Curl counter variables
left_reps = 0
right_reps = 0
left_position = None
right_position = None

In [42]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [43]:
# Setup Mediapipe instance
with mp_body.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while webcam.isOpened():
        ret, frame = webcam.read()
        if not ret:
            break

        # Convert to RGB
        eachframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        eachframe.flags.writeable = False
        results = pose.process(eachframe)
        eachframe.flags.writeable = True
        eachframe = cv2.cvtColor(eachframe, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark
        
            # Left Arm Coordinates
            left_shoulder = [landmarks[mp_body.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_body.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_body.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_body.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_body.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_body.PoseLandmark.LEFT_WRIST.value].y]
            left_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        
            # Right Arm Coordinates
            right_shoulder = [landmarks[mp_body.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_body.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_elbow = [landmarks[mp_body.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_body.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist = [landmarks[mp_body.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_body.PoseLandmark.RIGHT_WRIST.value].y]
            right_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        
            # Left Arm Curl Logic
            if left_angle > 160:
                left_position = "down"
            if left_angle < 30 and left_position == "down":
                left_position = "up"
                left_reps += 1
                speak(f"Left arm rep {left_reps}")
        
            # Right Arm Curl Logic
            if right_angle > 160:
                right_position = "down"
            if right_angle < 30 and right_position == "down":
                right_position = "up"
                right_reps += 1
                speak(f"Right arm rep {right_reps}")
        
        except:
            pass
        
        # Display Counters
        cv2.rectangle(eachframe, (5, 5), (220, 100), (100, 100, 100), -1)
        cv2.putText(eachframe, f'Left: {left_reps}', (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(eachframe, f'Right: {right_reps}', (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Draw landmarks
        mp_drawing.draw_landmarks(eachframe, results.pose_landmarks, mp_body.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(200, 100, 100), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(105, 105, 255), thickness=2, circle_radius=2))
        
        # Display the window
        cv2.imshow('Bicep Curl Counter', eachframe)
        
        # Quit with 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release webcam
webcam.release()
cv2.destroyAllWindows()
speak("Good job!")


Exception in thread Thread-41 (<lambda>):
Traceback (most recent call last):
  File "D:\coding\hereLiesAanaconda\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "D:\coding\hereLiesAanaconda\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\radna\AppData\Local\Temp\ipykernel_17792\3204626928.py", line 3, in <lambda>
  File "C:\Users\radna\AppData\Roaming\Python\Python312\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-43 (<lambda>):
Traceback (most recent call last):
  File "D:\coding\hereLiesAanaconda\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "D:\coding\hereLiesAanaconda\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\radna\AppData\Local\Temp\ipykernel_17792\3204626928.py", line 3, in <lambda>
  File "C:\Use